In [5]:
import pandas as pd
import numpy as np

In [6]:
# ===================== 统一文件命名常量区 =====================
# 输入：已清洗的基础数据集文件名（可在此统一修改）
INPUT_CLEANED_DATA = '1_cleaned_data.csv'
# 输出：排名法机制建模数据集保存文件名（可在此统一修改）
OUTPUT_RANK_MODELING_DATA = '2_RANK_modeling_data.csv'
# ==============================================================

In [7]:
# 重新加载清洗后的数据集
df = pd.read_csv(INPUT_CLEANED_DATA)

# 根据题目文档重新理解节目机制
print("=== 节目机制关键信息 ===")
print("1. 允许单周淘汰多名选手（文档第5点说明）")
print("2. 排名可能出现平局（多名选手相同排名）")
print("3. 某些周次可能无人被淘汰")

# 筛选排名法赛季: season 为 1、2 或 ≥28
rank_based_seasons = list(range(1, 3)) + list(range(28, 35))  # 赛季1-2, 28-34
rank_based_df = df[df['season'].isin(rank_based_seasons)]

print(f"\n排名法赛季数据形状: {rank_based_df.shape}")
print("包含赛季:", sorted(rank_based_df['season'].unique()))

=== 节目机制关键信息 ===
1. 允许单周淘汰多名选手（文档第5点说明）
2. 排名可能出现平局（多名选手相同排名）
3. 某些周次可能无人被淘汰

排名法赛季数据形状: (1265, 12)
包含赛季: [np.int64(1), np.int64(2), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34)]


In [8]:
# 按season和week分组，考虑节目实际机制
grouped = rank_based_df.groupby(['season', 'week'])

# 修正后的分组信息收集
group_info = []

for name, group in grouped:
    season, week = name
    n_players = len(group)
    
    # 只考虑未被淘汰的选手（was_eliminated为False）
    active_players = group[group['was_eliminated'] == False]
    n_active = len(active_players)
    
    # 获取本周被淘汰的选手
    eliminated_this_week = group[group['was_eliminated'] == True]
    n_eliminated = len(eliminated_this_week)
    
    # 检查评委排名：允许平局，但需验证排名范围
    judge_ranks = active_players['weekly_rank_by_avg'].dropna()
    
    # 修正排名验证：允许平局，检查排名是否在合理范围内
    rank_issues = []
    if len(judge_ranks) > 0:
        min_rank = judge_ranks.min()
        max_rank = judge_ranks.max()
        
        # 排名应该在1到n_active之间（允许平局）
        if min_rank < 1 or max_rank > n_active:
            rank_issues.append(f"排名范围异常: {min_rank}-{max_rank}, 预期1-{n_active}")
        
        # 检查排名是否单调（允许重复）
        unique_ranks = sorted(judge_ranks.unique())
        expected_progression = all(unique_ranks[i] <= unique_ranks[i+1] 
                                 for i in range(len(unique_ranks)-1))
        if not expected_progression:
            rank_issues.append("排名序列不单调")
    
    group_info.append({
        'season': season,
        'week': week,
        'n_total_players': n_players,
        'n_active_players': n_active,
        'n_eliminated_this_week': n_eliminated,
        'eliminated_players': list(eliminated_this_week['celebrity_name']),
        'active_players': list(active_players['celebrity_name']),
        'judge_ranks': list(active_players['weekly_rank_by_avg']),
        'rank_issues': rank_issues,
        'has_rank_data': len(judge_ranks) > 0
    })

# 转换为DataFrame
group_summary = pd.DataFrame(group_info)
print(f"\n总周次数量: {len(group_summary)}")


总周次数量: 99


In [9]:
# 重新定义验证标准
print("=== 修正后的数据验证 ===")

# 统计各类情况
total_groups = len(group_summary)
valid_groups = 0
issues_found = []

for i, group in group_summary.iterrows():
    season, week = group['season'], group['week']
    issues = []
    
    # 检查1: 是否有有效的排名数据
    if not group['has_rank_data']:
        issues.append("无有效排名数据")
    
    # 检查2: 排名范围是否合理
    if group['rank_issues']:
        issues.extend(group['rank_issues'])
    
    # 检查3: 淘汰人数是否合理（根据文档，允许0人或多人淘汰）
    # 不再将多人淘汰视为问题
    
    if issues:
        issues_found.append({
            'season': season, 
            'week': week, 
            'issues': issues,
            'n_eliminated': group['n_eliminated_this_week']
        })
    else:
        valid_groups += 1

print(f"有效周次: {valid_groups}/{total_groups} ({valid_groups/total_groups*100:.1f}%)")

if issues_found:
    print(f"\n发现问题的周次 ({len(issues_found)}个):")
    for issue in issues_found[:10]:  # 只显示前10个
        print(f"赛季 {issue['season']} 第 {issue['week']} 周: {', '.join(issue['issues'])}")
    if len(issues_found) > 10:
        print(f"... 以及另外 {len(issues_found)-10} 个周次")
else:
    print("所有周次数据验证通过")

=== 修正后的数据验证 ===
有效周次: 87/99 (87.9%)

发现问题的周次 (12个):
赛季 1 第 7.0 周: 无有效排名数据
赛季 1 第 8.0 周: 无有效排名数据
赛季 1 第 9.0 周: 无有效排名数据
赛季 1 第 10.0 周: 无有效排名数据
赛季 1 第 11.0 周: 无有效排名数据
赛季 2 第 9.0 周: 无有效排名数据
赛季 2 第 10.0 周: 无有效排名数据
赛季 2 第 11.0 周: 无有效排名数据
赛季 30 第 11.0 周: 无有效排名数据
赛季 31 第 11.0 周: 无有效排名数据
... 以及另外 2 个周次


In [10]:
# 处理平局排名的情况
def analyze_rank_ties(group_summary):
    """分析存在平局排名的情况"""
    tie_groups = []
    
    for i, group in group_summary.iterrows():
        if not group['has_rank_data']:
            continue
            
        judge_ranks = group['judge_ranks']
        # 检查是否有重复排名（平局）
        if len(judge_ranks) != len(set(judge_ranks)):
            tie_groups.append({
                'season': group['season'],
                'week': group['week'],
                'n_players': group['n_active_players'],
                'unique_ranks': len(set(judge_ranks)),
                'rank_distribution': pd.Series(judge_ranks).value_counts().to_dict()
            })
    
    return tie_groups

# 分析平局情况
tie_analysis = analyze_rank_ties(group_summary)
print(f"\n=== 平局排名分析 ===")
print(f"存在平局排名的周次: {len(tie_analysis)}")

if tie_analysis:
    for tie in tie_analysis[:5]:  # 显示前5个例子
        print(f"赛季 {tie['season']} 第 {tie['week']} 周: {tie['n_players']}名选手, "
              f"{tie['unique_ranks']}个不同排名")
        for rank, count in tie['rank_distribution'].items():
            if count > 1:
                print(f"  排名{rank}有{count}名选手")


=== 平局排名分析 ===
存在平局排名的周次: 72
赛季 1 第 1.0 周: 6名选手, 3个不同排名
  排名1.0有3名选手
  排名4.0有2名选手
赛季 2 第 1.0 周: 10名选手, 9个不同排名
  排名2.0有2名选手
赛季 2 第 2.0 周: 9名选手, 8个不同排名
  排名5.0有2名选手
赛季 2 第 3.0 周: 8名选手, 6个不同排名
  排名1.0有2名选手
  排名5.0有2名选手
赛季 2 第 4.0 周: 7名选手, 6个不同排名
  排名2.0有2名选手


In [11]:
# 创建最终可用的数据集
def create_modeling_dataset(group_summary, original_df):
    """创建用于建模的数据集"""
    
    # 筛选出有有效排名数据的周次
    valid_groups = group_summary[group_summary['has_rank_data'] == True]
    
    # 获取这些周次的数据
    valid_data = []
    for _, group in valid_groups.iterrows():
        season, week = group['season'], group['week']
        week_data = original_df[
            (original_df['season'] == season) & 
            (original_df['week'] == week) &
            (original_df['was_eliminated'] == False)  # 只使用未被淘汰选手的数据
        ].copy()
        
        if len(week_data) > 0:
            valid_data.append(week_data)
    
    if valid_data:
        modeling_df = pd.concat(valid_data, ignore_index=True)
        print(f"\n=== 最终建模数据集 ===")
        print(f"数据形状: {modeling_df.shape}")
        print(f"包含周次: {len(valid_groups)}")
        print(f"选手-周次记录数: {len(modeling_df)}")
        
        # 添加平局标记
        modeling_df['has_tie'] = modeling_df.groupby(['season', 'week'])['weekly_rank_by_avg'].transform(
            lambda x: len(x) != len(set(x))
        )
        
        return modeling_df
    else:
        print("无有效数据可用于建模")
        return pd.DataFrame()

# 生成最终数据集
modeling_dataset = create_modeling_dataset(group_summary, rank_based_df)

if not modeling_dataset.empty:
    # 显示数据集统计信息
    print("\n数据集统计:")
    print(f"- 季节数量: {modeling_dataset['season'].nunique()}")
    print(f"- 总周次: {modeling_dataset[['season', 'week']].drop_duplicates().shape[0]}")
    print(f"- 包含平局的周次: {modeling_dataset[modeling_dataset['has_tie']].groupby(['season', 'week']).ngroups}")
    
    # 保存修正后的数据集
    modeling_dataset.to_csv(OUTPUT_RANK_MODELING_DATA, index=False)
    print(f"\n修正后的数据集已保存为: {OUTPUT_RANK_MODELING_DATA}")


=== 最终建模数据集 ===
数据形状: (780, 12)
包含周次: 87
选手-周次记录数: 780

数据集统计:
- 季节数量: 9
- 总周次: 87
- 包含平局的周次: 72

修正后的数据集已保存为: 2_RANK_modeling_data.csv
